https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=400

Crime reports?


In [ ]:
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = path.join(PROFILE_PATH, 'Collections/crimes');

function scrapeAlert(ID) {
    if(fs.existsSync(path.join(project, 'maricopa-alert-' + ID + '.json'))) {
        return;
    }
    return client
        .url('https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=' + ID)
        .getAllXPath({
            time: '//*[@class = "single"]//*[@class = "date"]//text()',
            title: '//*[contains(@class, "single")]//h3//text()',
            content: '//*[@class = "single"]//*[@class = "content"]//text()'
        })
        .then(r => {
            fs.writeFileSync(path.join(project, 'maricopa-alert-' + ID + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
}
module.exports = scrapeAlert;

if(typeof $$ !== 'undefined') {
    $$.async();
    var IDs = Array.from(Array(500).keys()).map(i => );
    multiCrawl(IDs, 'crime reports')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


multi crawl?


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var CONNECTIONS = 10;

function multiCrawl(arr, search) {
    var contexts = [];
    return importer.runAllPromises(Array.from(Array(CONNECTIONS).keys()).map((s, i) => resolve => {
        const client = runSeleniumCell(false, false);
        const ctx = {client};
        const funcs = importer.import([
            'decrypt password',
            'all elements xpath',
            'get all elements until',
            'resize selenium window',
            'click spa link'
        ], ctx);
        contexts[i] = Object.assign(ctx, funcs);
        return client
            .catch(e => console.log(e))
            .then(r => resolve())
    }))
        .then(() => importer.import('only one window', contexts[0])())
        .then(() => {
            const promises = [];
            for(var j = 0; j < arr.length / CONNECTIONS; j++) {
                promises.push((j => resolve => Promise.all(contexts.map((ctx, i) => {
                    const callback = importer.import(search, ctx);
                    if(arr.length < j * CONNECTIONS + i) {
                        return;
                    }
                    return callback(arr[j * CONNECTIONS + i]);
                })).then(r => resolve(r)))(j));
            }
            return importer.runAllPromises(promises);
        })
}
module.exports = multiCrawl;

